In [4]:
from pyspark.sql.functions import col

from petastorm.spark import SparkDatasetConverter, make_spark_converter

import io
import numpy as np
import tensorflow as tf
from PIL import Image
from petastorm import TransformSpec
from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

/home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem
2023-04-10 16:26:33.770359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from sparkdl import HorovodRunner

ModuleNotFoundError: No module named 'sparkdl'

In [15]:
!HOROVOD_WITH_TENSORFLOW=1 pip install horovod[tensorflow]

/bin/bash: /home/spark/miniconda3/envs/recommend/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached horovod-0.27.0.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [284 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/horovod
      copying horovod/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod
      creating build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/elastic.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/basics.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/exceptions.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/process_sets.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/util.py -> build/lib

Failed to build horovod
  Running setup.py install for horovod ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for horovod did not run successfully.
  │ exit code: 1
  ╰─> [288 lines of output]
      running install
      /home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/horovod
      copying horovod/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod
      creating build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/elastic.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copyin

In [21]:
!HOROVOD_WITH_TENSORFLOW=1 pip install horovod[tensorflow]

/bin/bash: /home/spark/miniconda3/envs/recommend/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached horovod-0.27.0.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [320 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/horovod
      copying horovod/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod
      creating build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/elastic.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/basics.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/exceptions.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/process_sets.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/util.py -> build/lib

Failed to build horovod
  Running setup.py install for horovod ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for horovod did not run successfully.
  │ exit code: 1
  ╰─> [324 lines of output]
      running install
      /home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/horovod
      copying horovod/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod
      creating build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/__init__.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copying horovod/common/elastic.py -> build/lib.linux-x86_64-cpython-39/horovod/common
      copyin